In [20]:
# Setup: Add project root to path and import utilities
import sys
sys.path.insert(0, '..')

from utils import get_input
import numpy as np
from math import prod

# Load input data for day 9
data = get_input(9, example=False)

print(f"Loaded {len(data)} lines of input")
print("Sample data:")
for line in data[:5]:
    print(f"  {line}")

Loaded 496 lines of input
Sample data:
  97926,50400
  97926,51618
  98025,51618
  98025,52819
  97711,52819


In [31]:
# Parse input data
grid_data = [tuple(int(coord) for coord in line.split(',')) for line in data]
print(f"Parsed {len(grid_data)} coordinate pairs")

# Step 1: Define red tiles (boundary points)
red_tiles = set(grid_data)
print(f"\nStep 1: Red tiles = {len(red_tiles)}")

# Step 2: Define edge green tiles (straight lines between consecutive red tiles)
def create_edge_tiles(red_tiles_list):
    """Create green tiles along edges between consecutive red tiles."""
    edge_tiles = set()
    
    for i in range(len(red_tiles_list)):
        x1, y1 = red_tiles_list[i]
        x2, y2 = red_tiles_list[(i + 1) % len(red_tiles_list)]
        
        # Add tiles along straight line (horizontal or vertical)
        if x1 == x2:  # Vertical line
            for y in range(min(y1, y2), max(y1, y2) + 1):
                edge_tiles.add((x1, y))
        elif y1 == y2:  # Horizontal line
            for x in range(min(x1, x2), max(x1, x2) + 1):
                edge_tiles.add((x, y1))
    
    # Remove red tiles from edge tiles (they're not green)
    edge_tiles -= red_tiles
    return edge_tiles

edge_tiles = create_edge_tiles(grid_data)
border_B = red_tiles | edge_tiles
print(f"Step 2: Edge green tiles = {len(edge_tiles)}")
print(f"        Border B (red + green) = {len(border_B)}")

# Step 3: Find smallest rectangle OB that fits all red tiles
xs = [x for x, y in red_tiles]
ys = [y for x, y in red_tiles]
OB_min_x, OB_max_x = min(xs), max(xs)
OB_min_y, OB_max_y = min(ys), max(ys)
print(f"\nStep 3: Outer Border OB = ({OB_min_x},{OB_min_y}) to ({OB_max_x},{OB_max_y})")
print(f"        OB dimensions: {OB_max_x - OB_min_x + 1} x {OB_max_y - OB_min_y + 1}")

# Step 4: Ray-casting for point-in-polygon
def is_point_inside_polygon(x, y, polygon_vertices):
    """Ray-casting algorithm."""
    inside = False
    n = len(polygon_vertices)
    j = n - 1
    for i in range(n):
        xi, yi = polygon_vertices[i]
        xj, yj = polygon_vertices[j]
        if ((yi > y) != (yj > y)) and (x < (xj - xi) * (y - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

print(f"\nStep 4: Ray-casting function defined for polygon containment")

# Step 5: Overlap function
def Overlap(r1_x1, r1_y1, r1_x2, r1_y2, r2_x1, r2_y1, r2_x2, r2_y2):
    """Check if two rectangles overlap."""
    r1_min_x, r1_max_x = min(r1_x1, r1_x2), max(r1_x1, r1_x2)
    r1_min_y, r1_max_y = min(r1_y1, r1_y2), max(r1_y1, r1_y2)
    r2_min_x, r2_max_x = min(r2_x1, r2_x2), max(r2_x1, r2_x2)
    r2_min_y, r2_max_y = min(r2_y1, r2_y2), max(r2_y1, r2_y2)
    
    # No overlap if separated
    if r1_max_x < r2_min_x or r2_max_x < r1_min_x:
        return False
    if r1_max_y < r2_min_y or r2_max_y < r1_min_y:
        return False
    return True

print(f"Step 5: Overlap(R, R') function defined")

# Step 6: IsWithinBorder - check if rectangle contains only red/green tiles
def IsWithinBorder(x1, y1, x2, y2):
    """
    Check if rectangle contains only red and green tiles.
    According to the problem:
    - All tiles inside the red/green boundary loop are green
    - Rectangle corners must be red tiles (enforced by main loop)
    - Rectangle can include any red/green tiles (border or interior)
    
    So we just need to ensure all 4 corners are inside or on the polygon boundary.
    """
    # Normalize coordinates
    min_x, max_x = min(x1, x2), max(x1, x2)
    min_y, max_y = min(y1, y2), max(y1, y2)
    
    # Check if all 4 corners are inside or on the polygon boundary
    corners = [(min_x, min_y), (min_x, max_y), (max_x, min_y), (max_x, max_y)]
    for cx, cy in corners:
        # Corner must be on the boundary (red/green tile) or inside (green tile)
        if (cx, cy) not in border_B and not is_point_inside_polygon(cx, cy, grid_data):
            return False
    
    return True

print(f"Step 6 & 7: IsWithinBorder(R) function defined (all interior tiles are green)")

# Find largest rectangle
from math import prod

print(f"\n{'='*60}")
print(f"Finding largest rectangle with red tile corners within border...")
print(f"{'='*60}")

red_tiles_list = list(red_tiles)
n = len(red_tiles_list)
print(f"Checking {n * (n - 1) // 2} possible rectangle pairs...")

largest_area = 0
best_rectangle = None
checked = 0

for i in range(n):
    if i % 50 == 0:
        print(f"Progress: {i}/{n} red tiles checked, largest area so far: {largest_area}")
    
    x1, y1 = red_tiles_list[i]
    for j in range(i + 1, n):
        x2, y2 = red_tiles_list[j]
        
        # Calculate area
        width = abs(x2 - x1)
        height = abs(y2 - y1)
        area = width * height
        
        # Skip if not better than current best
        if area <= largest_area:
            continue
        
        checked += 1
        if IsWithinBorder(x1, y1, x2, y2):
            largest_area = area
            best_rectangle = (x1, y1, x2, y2)
            print(f"  New largest: ({x1},{y1}) to ({x2},{y2}), area = {area}")

print(f"\n{'='*60}")
print(f"RESULT: Largest rectangle area = {largest_area}")
if best_rectangle:
    print(f"Rectangle: ({best_rectangle[0]},{best_rectangle[1]}) to ({best_rectangle[2]},{best_rectangle[3]})")
    print(f"Dimensions: {abs(best_rectangle[2] - best_rectangle[0])} x {abs(best_rectangle[3] - best_rectangle[1])}")
print(f"Checked {checked} candidate rectangles")
print(f"{'='*60}")

Parsed 496 coordinate pairs

Step 1: Red tiles = 496
Step 2: Edge green tiles = 586138
        Border B (red + green) = 586634

Step 3: Outer Border OB = (1515,1666) to (98234,98094)
        OB dimensions: 96720 x 96429

Step 4: Ray-casting function defined for polygon containment
Step 5: Overlap(R, R') function defined
Step 6 & 7: IsWithinBorder(R) function defined (all interior tiles are green)

Finding largest rectangle with red tile corners within border...
Checking 122760 possible rectangle pairs...
Progress: 0/496 red tiles checked, largest area so far: 0
  New largest: (63664,3712) to (62790,96155), area = 80795182
  New largest: (63664,3712) to (37537,95655), area = 2402194761
  New largest: (87985,21101) to (12232,79338), area = 4411627461
  New largest: (87985,21101) to (11436,79338), area = 4457984113
  New largest: (13931,82043) to (86222,19050), area = 4553826963
Progress: 50/496 red tiles checked, largest area so far: 4553826963
  New largest: (82264,14882) to (17514,8535